In [ ]:
# Prompt:
# 这是文件的详细内容，请注意文件内容包含每年十二个月份的均值，我需要的是每年的均值，所以要把一年的十二个月份的均值计算出来
# 这是国家边界数据文件路径，“/Users/xuchengwei/Desktop/汇丰论文/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp”
# 现在我有在这个文件夹里面三个文件，包含各种气候数据
# 请帮我批量处理好每个国家的数据，然后每个国家有不同年份的气候面板数据，
# 文件夹是这个“/Users/xuchengwei/Desktop/汇丰论文/气候数据”，
# 里面有三个文件，分别是“/Users/xuchengwei/Desktop/汇丰论文/气候数据/data_0.nc”，“/Users/xuchengwei/Desktop/汇丰论文/气候数据/data_1.nc”，“/Users/xuchengwei/Desktop/汇丰论文/气候数据/data_2.nc”
# 最后给我输出一个csv包含每个国家有不同年份的气候面板数据

In [ ]:
import xarray as xr

# 读取文件
ds_0 = xr.open_dataset("/Users/xuchengwei/Desktop/汇丰论文/气候数据/data_0.nc")
ds_1 = xr.open_dataset("/Users/xuchengwei/Desktop/汇丰论文/气候数据/data_1.nc")
ds_2 = xr.open_dataset("/Users/xuchengwei/Desktop/汇丰论文/气候数据/data_2.nc")

# 查看文件元数据（变量、维度、坐标等）
print(ds_0,ds_1,ds_2)  # 输出文件结构

In [6]:
import geopandas as gpd
import xarray as xr
import pandas as pd
import numpy as np
import regionmask
from tqdm import tqdm
from joblib import Parallel, delayed
import os
import logging

# 设置日志
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 文件路径
shp_path = "/Users/xuchengwei/Desktop/汇丰论文/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp"
data_folder = "/Users/xuchengwei/Desktop/汇丰论文/气候数据"
nc_files = [
    "data_0.nc",
    "data_1.nc",
    "data_2.nc"
]
output_csv = "/Users/xuchengwei/Desktop/汇丰论文/climate_panel_data.csv"

# 所有气候变量
climate_vars = ['stl1', 'sro', 'uvb', 'sf', 'tp', 'lai_hv', 'sp', 'sd', 'tcc', 't2m', 'si10']

# 读取国家边界数据
logging.info("读取国家边界数据...")
gdf = gpd.read_file(shp_path)
countries = gdf[['ADM0_A3', 'geometry']].to_crs(epsg=4326)

# 测试模式：可选，限制国家数量以调试
# countries = countries[countries['ADM0_A3'].isin(['USA', 'CHN'])]  # 取消注释以测试少量国家

# 定义函数：处理单个年份的NetCDF数据
def process_year(year, nc_file, countries_gdf, variables):
    logging.info(f"处理年份 {year}，文件 {nc_file}...")
    try:
        ds = xr.open_dataset(nc_file)
        # 选择该年份的数据并计算年均值
        yearly_ds = ds.sel(valid_time=ds.valid_time.dt.year == year).mean(dim='valid_time')
        
        # 动态检测经纬度维度名称
        lon_name = 'longitude' if 'longitude' in yearly_ds.dims else 'lon'
        lat_name = 'latitude' if 'latitude' in yearly_ds.dims else 'lat'
        logging.info(f"检测到的经纬度维度：{lon_name}, {lat_name}")
        
        # 重命名维度以适配regionmask
        yearly_ds = yearly_ds.rename({lon_name: 'lon', lat_name: 'lat'})
        
        # 使用regionmask创建国家掩膜
        mask = regionmask.from_geopandas(countries_gdf, names='ADM0_A3')
        
        country_data = []
        for var in variables:
            if var in yearly_ds:
                logging.info(f"处理变量 {var}...")
                # 应用掩膜并计算每个国家的均值
                masked_data = mask.mask(yearly_ds[var])
                for idx, country_iso in enumerate(countries_gdf['ADM0_A3']):
                    country_mask = masked_data == idx
                    if country_mask.any():
                        mean_value = yearly_ds[var].where(country_mask).mean().values
                        if not np.isnan(mean_value):
                            country_data.append({
                                'ISO3': country_iso,
                                'Year': year,
                                'Variable': var,
                                'Mean': float(mean_value)
                            })
        ds.close()
        return country_data
    except Exception as e:
        logging.error(f"处理年份 {year}，文件 {nc_file} 失败: {str(e)}")
        return []

# 主处理逻辑
logging.info("开始处理NetCDF文件...")
all_data = []

for nc_file in nc_files:
    ds = xr.open_dataset(os.path.join(data_folder, nc_file))
    years = np.unique(ds['valid_time'].dt.year.values)
    ds.close()
    
    # 并行处理年份
    results = Parallel(n_jobs=4)(
        delayed(process_year)(year, os.path.join(data_folder, nc_file), countries, climate_vars)
        for year in tqdm(years, desc=f"处理文件 {nc_file}")
    )
    
    # 展平结果
    for result in results:
        all_data.extend(result)

# 转换为DataFrame并重塑为宽格式
logging.info("生成面板数据...")
panel_df = pd.DataFrame(all_data)
if not panel_df.empty:
    panel_df = panel_df.pivot_table(
        index=['ISO3', 'Year'],
        columns='Variable',
        values='Mean',
        aggfunc='first'
    ).reset_index()
else:
    logging.warning("没有生成有效数据，输出为空DataFrame")
    panel_df = pd.DataFrame(columns=['ISO3', 'Year'] + climate_vars)

# 保存到CSV
logging.info(f"保存结果到 {output_csv}...")
panel_df.to_csv(output_csv, index=False)
logging.info("处理完成！")

2025-05-27 19:12:49,724 - INFO - 读取国家边界数据...
2025-05-27 19:12:49,837 - INFO - 开始处理NetCDF文件...
处理文件 data_2.nc: 100%|██████████| 36/36 [07:53<00:00, 13.16s/it]
2025-05-27 19:28:14,690 - INFO - 生成面板数据...
2025-05-27 19:28:15,003 - INFO - 保存结果到 /Users/xuchengwei/Desktop/汇丰论文/climate_panel_data.csv...
2025-05-27 19:28:15,129 - INFO - 处理完成！
